In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/LOOCV_2models_withmasks/loo_temp_Anthisnes_Chateau_de_Xhos_Camera_1_HIT.zip /content
!unzip /content/loo_temp_Anthisnes_Chateau_de_Xhos_Camera_1_HIT.zip -d /content > /dev/null

```markdown
# -----------------------------------------------------------------------------
# Script Structure Overview
# -----------------------------------------------------------------------------
# 1. Configuration: Import modules, define paths, set random seed, and list candidate gîtes.
# 2. Held-out Selection: Randomly choose one gîte for validation to ensure reproducibility.
# 3. Directory Preparation: Discover classes and create corresponding validation subdirectories.
# 4. File Movement Function: Encapsulate logic to move images and masks for a specific class.
# 5. Execution Loop: Iterate over all classes, move matching files, and report counts.
# 6. Summary: Print the total number of moved pairs, indicating completion of the split.
```

In [ ]:
import os
import random
import shutil

# -----------------------------------------------------------------------------
# Configuration
# -----------------------------------------------------------------------------
# Define the base directory containing training and validation data subfolders.
base_path = "/content/loo_temp_Anthisnes_Chateau_de_Xhos_Camera_1_HIT"
train_img_dir = os.path.join(base_path, "train_images")  # Path to training images
train_mask_dir = os.path.join(base_path, "train_masks")   # Path to training masks
val_img_dir = os.path.join(base_path, "val_images")      # Destination for validation images
val_mask_dir = os.path.join(base_path, "val_masks")     # Destination for validation masks

# Set a fixed random seed to ensure reproducibility of the held-out selection.
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

# List of candidate gîte identifiers. One will be randomly selected for validation.
chosen_gites = [
    'Pont_de_Bousval_Photos_2022_PHOTO',
    'Modave_Camera_3_toiture_PHOTO',
    'Bornival_PHOTO_2023CAM04',
    'Pont_de_Bousval_Photos_2023_PHOTO_WK6HDBOUSVAL',
    'Pont_de_Bousval_Photos_2023_PHOTO_2022CAM12',
    'Pont_de_Bousval_Photos_2023_PHOTO_2023CAM06',
    'Bornival_PHOTO_2023CAM03',
    'Chaumont_Gistoux_Camera_2',
    'Chaumont_Gistoux_Camera_1',
    'Pont_de_Bousval_Photos_2023_PHOTO_2023CAM05',
    'Jenneret_Camera_1_PHOTO',
    'Modave_Camera_plancher_PHOTO'
]

# Randomly choose one gîte to hold out for the validation set.
held_out_gite = random.choice(chosen_gites)
print(f"Holding out gîte for validation: {held_out_gite}")
print(f"Reproducible with seed: {RANDOM_SEED}")

# -----------------------------------------------------------------------------
# Prepare validation directories per class
# -----------------------------------------------------------------------------
# Identify classes by listing subdirectories in the training image directory.
classes = [d for d in os.listdir(train_img_dir)
           if os.path.isdir(os.path.join(train_img_dir, d))]

# Ensure the base validation directories exist.
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(val_mask_dir, exist_ok=True)

# Create class-specific subdirectories inside the validation folders.
for cls in classes:
    os.makedirs(os.path.join(val_img_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_mask_dir, cls), exist_ok=True)


def move_class_images_and_masks(cls, gite_name):
    """
    Move all images and their corresponding masks belonging to a specific class
    and matching the held-out gîte identifier into the validation directories.

    Args:
        cls (str): Name of the class subdirectory.
        gite_name (str): Identifier of the held-out gîte to match in filenames.

    Returns:
        int: Number of image-mask pairs successfully moved.
    """
    src_img_cls = os.path.join(train_img_dir, cls)
    src_mask_cls = os.path.join(train_mask_dir, cls)
    dst_img_cls = os.path.join(val_img_dir, cls)
    dst_mask_cls = os.path.join(val_mask_dir, cls)
    moved = 0

    for fname in sorted(os.listdir(src_img_cls)):
        if gite_name in fname:
            img_src = os.path.join(src_img_cls, fname)
            base, _ = os.path.splitext(fname)
            mask_name = f"{base}_mask.png"
            mask_src = os.path.join(src_mask_cls, mask_name)

            if os.path.exists(mask_src):
                # Move both image and mask to the validation directory
                shutil.move(img_src, os.path.join(dst_img_cls, fname))
                shutil.move(mask_src, os.path.join(dst_mask_cls, mask_name))
                moved += 1
            else:
                # Warn if a mask is expected but missing
                print(f"Warning: Mask not found for {cls} image: {fname}")

    return moved

# -----------------------------------------------------------------------------
# Execution: Move files for all classes and report summary
# -----------------------------------------------------------------------------
total_moved = 0
for cls in classes:
    moved_count = move_class_images_and_masks(cls, held_out_gite)
    print(f"Class '{cls}': moved {moved_count} image-mask pairs")
    total_moved += moved_count

print(f"Total moved {total_moved} image-mask pairs to validation.")


# LOO CV Masked-Bat Classifier (EfficientNet-B0 Setup)

This repository implements a **leave-one-out cross-validation (LOO CV)** pipeline to train and evaluate a binary image classifier that distinguishes bats from background. It uses masked inputs, data augmentations with MixUp/CutMix, a custom focal loss, and a partially frozen EfficientNet-B0 backbone.

---

## Features

- **Leave-one-out cross-validation** for robust evaluation on small datasets  
- **Masked inputs**: per-image binary masks to isolate foreground  
- **Data augmentations**: random crop, flip, rotation, color jitter, perspective, blur, erasing  
- **MixUp/CutMix** for enhanced generalization  
- **Custom Focal Loss** to address class imbalance  
- **Partial freezing** of EfficientNet-B0 feature blocks for efficient transfer learning  
- **Automatic threshold selection** via precision–recall curve  
- **ROC curve plotting** and AUC computation  
- **End-to-end inference** with mask application and decision threshold  

---

## Directory Structure

```
/content/loo_temp_<site>_<camera>Camera<fold>/
├── train_images/
│   ├── background/
│   └── bats/
├── train_masks/
│   ├── background/
│   └── bats/
├── val_images/
│   ├── background/
│   └── bats/
├── val_masks/
│   ├── background/
│   └── bats/
├── test_images/
│   ├── background/
│   └── bats/
└── test_masks/
    ├── background/
    └── bats/
```
Note:
Each image in `*_images/` must have a corresponding mask in `*_masks/` named `<image_basename>_mask.png`. Folders are organized by class (`background` / `bats`).

## Key Components

1. **Configuration**  
   - Define dataset directories, output paths, random seed, batch size, number of epochs, and learning rate  
   - Set up device (GPU/CPU), mixed-precision (AMP), and reproducibility  

2. **Transforms & Augmentation**  
   - **Image Transforms**  
     - `RandomResizedCrop(224)`, `RandomHorizontalFlip`, `RandomRotation(15)`  
     - `ColorJitter`, `RandomPerspective`, `GaussianBlur`, `RandomErasing`  
     - Normalize to ImageNet mean/std  
   - **Mask Transforms**  
     - Resize to 224×224 (nearest neighbour)  
     - Binarize mask to `[0, 1]`  
   - **MixUp/CutMix**  
     - Batch-level mixing with configurable α parameters  

3. **Dataset Class**  
   - **`MaskedImageDataset`**  
     - Loads image and mask pairs, applies transforms, multiplies image by mask  
     - Returns `(masked_image, label)`  

4. **Data Splits & Loaders**  
   - **Training**: Balanced sampling via `WeightedRandomSampler`, data prefetching  
   - **Validation/Test**: Sequential loading, larger batch size  

5. **Loss Function**  
   - **Focal Loss**  
     - α = `[0.05, 0.95]`, γ = `2.0`  
     - Emphasizes hard examples, down-weights easy ones  

6. **Model & Freezing**  
   - Pretrained **EfficientNet-B0** backbone  
   - Freeze first two-thirds of feature blocks to retain pretrained representations  
   - Replace classifier head with `Dropout(0.5) → Linear(in_features → 2)`  
   - Optionally wrap in `torch.compile()` for PyTorch ≥ 2.0  

7. **Optimizer & Scheduler**  
   - **Optimizer**: `AdamW`, learning rate = `3 × 10⁻⁵`  
   - **Scheduler**: `CosineAnnealingLR` over total training steps  

8. **Training & Validation Loop**  
   - **Training**  
     - Mixed precision (`autocast`, `GradScaler`)  
     - Apply MixUp/CutMix  
     - Forward/backward pass, optimizer step, scheduler step  
   - **Validation**  
     - Forward only  
     - Compute validation loss and macro F1  
   - **Early Stopping**  
     - Stop if no F1 improvement for 3 consecutive epochs  
   - **Checkpointing**  
     - Save best model weights and validation logits/labels  

9. **Threshold Optimization & ROC Analysis**  
   - Load saved validation logits and labels  
   - Compute precision–recall curve; select threshold where precision ≈ recall  
   - Compute ROC curve and AUC; plot with threshold marker  

10. **Inference on Test Set**  
    - Reload best model and optimal threshold  
    - For each test image:  
      1. Apply mask (if available)  
      2. Forward pass → probability of “bat”  
      3. Apply threshold → predict “bats” vs “background”  
      4. Log filename, predicted class, probability, mask usage  

11. **Final Evaluation**  
    - Aggregate test logits and labels  
    - Compute precision, recall, macro F1, ROC AUC  
    - Print classification report (per-class metrics)  
    - Plot final ROC curve for test set  

## Hardware Requirements

- **GPU** recommended (e.g., NVIDIA T4 on Colab) for mixed precision  
- Falls back to CPU if no CUDA device is available  

## Outputs

- `best.pth`: Checkpoint with highest validation F1  
- `blob.pt`: Validation logits & labels for threshold tuning  
- `thr.txt`: Selected optimal threshold value  
- Inline ROC plots for validation and test phases  

## Usage Notes

- Adjust `NUM_EPOCHS`, `BATCH_SIZE`, and `LEARNING_RATE` to match dataset size and hardware  
- Ensure mask files are correctly named and binary  
- For full LOO CV, wrap script in an outer loop over folds  
- Enable `torch.compile()` on PyTorch ≥ 2.0 for additional speed gains  

In [ ]:
"""
Loo CV masked-bat classifier with train/val/test splits, augmentations, MixUp/CutMix,
FocalLoss, EfficientNet-B0 head freezing, and inference.
"""
import os
import random
import shutil
from collections import Counter

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from torchvision.transforms import InterpolationMode
from PIL import Image
from tqdm.auto import tqdm
from timm.data.mixup import Mixup
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.cuda.amp import autocast, GradScaler
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    precision_recall_curve, roc_curve, auc,
    classification_report
)
import matplotlib.pyplot as plt

# --------------------------
# CONFIGURATION
# --------------------------
DATA_DIR       = "/content/loo_temp_Anthisnes_Chateau_de_Xhos_Camera_1_HIT"
TRAIN_IMG_DIR  = os.path.join(DATA_DIR, "train_images")
TRAIN_MASK_DIR = os.path.join(DATA_DIR, "train_masks")
VAL_IMG_DIR    = os.path.join(DATA_DIR, "val_images")
VAL_MASK_DIR   = os.path.join(DATA_DIR, "val_masks")
TEST_IMG_DIR   = os.path.join(DATA_DIR, "test_images")
TEST_MASK_DIR  = os.path.join(DATA_DIR, "test_masks")


# --------------------------
# HYPERPARAMS
# --------------------------
BATCH_SIZE      = 32
NUM_EPOCHS      = 10
LEARNING_RATE   = 3e-5
OUTPUT_DIR      = "/content/efficientnet_loocv_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --------------------------
# DEVICE & AMP
# --------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('high')
try:
    from torch.amp import autocast, GradScaler
    AMP_KW = dict(device_type='cuda')
except ImportError:
    from torch.cuda.amp import autocast, GradScaler
    AMP_KW = {}
scaler = GradScaler()

# --------------------------
# TRANSFORMS + MIXUP/CUTMIX
# --------------------------
weights = EfficientNet_B0_Weights.IMAGENET1K_V1
normalize = transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.5,1.0), ratio=(0.9,1.1), interpolation=InterpolationMode.BICUBIC),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.RandomPerspective(distortion_scale=0.3, p=0.3),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1,2.0)),
    transforms.ToTensor(),
    normalize,
    transforms.RandomErasing(scale=(0.02,0.2), ratio=(0.3,3.3), p=0.25),
])
val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize,
])
mask_transform = transforms.Compose([
    transforms.Resize((224,224), interpolation=Image.NEAREST),
    transforms.ToTensor(),
    transforms.Lambda(lambda t: (t>0.5).float()),
])
mixup_fn = Mixup(
    mixup_alpha=0.2, cutmix_alpha=1.0,
    prob=0.5, switch_prob=0.5,
    mode='batch', label_smoothing=0.0,
    num_classes=2
)

# --------------------------
# DATASET
# --------------------------
class MaskedImageDataset(Dataset):
    EXT = {'.jpg','.jpeg','.png','.bmp','.tiff'}
    def __init__(self, img_root, mask_root, img_tfm=None, mask_tfm=None):
        self.img_root, self.mask_root = img_root, mask_root
        self.img_tfm, self.mask_tfm = img_tfm, mask_tfm
        self.classes = sorted(d.name for d in os.scandir(img_root) if d.is_dir())
        self.cls2idx = {c:i for i,c in enumerate(self.classes)}
        self.samples, self.targets = [], []
        for c in self.classes:
            for fn in os.listdir(os.path.join(img_root, c)):
                ext = os.path.splitext(fn)[1].lower()
                if ext not in self.EXT: continue
                ip = os.path.join(img_root, c, fn)
                mp = os.path.join(mask_root, c, os.path.splitext(fn)[0] + '_mask.png')
                mp = mp if os.path.isfile(mp) else None
                self.samples.append((ip, mp))
                self.targets.append(self.cls2idx[c])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, mask_path = self.samples[idx]
        label = self.targets[idx]

        img = Image.open(img_path).convert('RGB')
        if self.img_tfm:
            img = self.img_tfm(img)

        if mask_path:
            m = Image.open(mask_path).convert('L')
            m = self.mask_tfm(m) if self.mask_tfm else m
        else:
            m = torch.ones_like(img[:1, :, :])

        return img * m, label

# --------------------------
# DATASPLITS & LOADERS
# --------------------------
train_set  = MaskedImageDataset(TRAIN_IMG_DIR, TRAIN_MASK_DIR, img_tfm=train_transform, mask_tfm=mask_transform)
val_set    = MaskedImageDataset(VAL_IMG_DIR,   VAL_MASK_DIR,   img_tfm=val_transform,   mask_tfm=mask_transform)
test_set   = MaskedImageDataset(TEST_IMG_DIR,  TEST_MASK_DIR,  img_tfm=val_transform,   mask_tfm=mask_transform)

# balanced sampler for training
targs = torch.tensor(train_set.targets)
cnts  = torch.tensor([Counter(targs.tolist())[i] for i in range(len(train_set.classes))], dtype=torch.float)
cls_w = 1. / cnts
cls_w /= cls_w.sum()
samp_w = cls_w[targs]
sampler = WeightedRandomSampler(samp_w, num_samples=len(train_set)*2, replacement=True)

num_w = os.cpu_count() or 2
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, sampler=sampler,
                          num_workers=num_w, pin_memory=True,
                          persistent_workers=True, prefetch_factor=4)
val_loader   = DataLoader(val_set,   batch_size=BATCH_SIZE*2, shuffle=False,
                          num_workers=num_w, pin_memory=True,
                          persistent_workers=True)
test_loader  = DataLoader(test_set,  batch_size=BATCH_SIZE*2, shuffle=False,
                          num_workers=num_w, pin_memory=True,
                          persistent_workers=True)

# --------------------------
# LOSS
# --------------------------
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = torch.tensor(alpha, dtype=torch.float32) if isinstance(alpha,(list,tuple)) else alpha
        self.gamma, self.reduction = gamma, reduction
    def forward(self, logits, targets):
        ce = F.cross_entropy(logits, targets, reduction='none')
        pt = torch.exp(-ce)
        if isinstance(self.alpha, torch.Tensor):
            a = self.alpha.to(logits.device)[targets] if targets.dtype in (torch.int64,torch.int32) else self.alpha.mean()
        else: a = self.alpha
        loss = a * (1-pt)**self.gamma * ce
        return loss.mean() if self.reduction=='mean' else loss.sum()

# --------------------------
# MODEL + FREEZE
# --------------------------
# load pretrained EfficientNet-B0
weights = EfficientNet_B0_Weights.IMAGENET1K_V1
base = efficientnet_b0(weights=weights).to(DEVICE)

# freeze first 2/3 of feature blocks
features = list(base.features)
freeze_upto = int(len(features) * 2 / 3)
for i, block in enumerate(features):
    if i < freeze_upto:
        for p in block.parameters():
            p.requires_grad = False

# replace classifier head
in_feats = base.classifier[1].in_features
base.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(in_feats, 2)
).to(DEVICE)

try:
    model = torch.compile(base)
except:
    model = base

# optimizer & scheduler
loss_fn = FocalLoss(alpha=[0.05,0.95], gamma=2.0).to(DEVICE)
opt     = AdamW(model.parameters(), lr=LEARNING_RATE)
sch     = CosineAnnealingLR(opt, T_max=NUM_EPOCHS * len(train_loader))

# --------------------------
# TRAIN/VALID LOOP
# --------------------------
best_f1, noimp, patience = 0.0, 0, 3
for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    train_loss = 0.0
    for x, y in tqdm(train_loader, desc=f"Train {epoch}/{NUM_EPOCHS}"):
        x, y = x.to(DEVICE), y.to(DEVICE)
        x, y = mixup_fn(x, y)
        with autocast(**AMP_KW):
            logits = model(x)
            loss = loss_fn(logits, y)
        opt.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()
        sch.step()
        train_loss += loss.item()

    avg_tr = train_loss / len(train_loader)

    # validation
    model.eval()
    vloss, preds, labs = 0.0, [], []
    with torch.no_grad():
        for x, y in tqdm(val_loader, desc="Valid"):
            x, y = x.to(DEVICE), y.to(DEVICE)
            with autocast(**AMP_KW):
                out = model(x)
                loss = loss_fn(out, y)
            vloss += loss.item()
            preds.append(out.cpu())
            labs.append(y.cpu())

    val_logits = torch.cat(preds)
    val_labs   = torch.cat(labs)
    avg_val    = vloss / len(val_loader)
    f1m        = f1_score(val_labs, val_logits.argmax(dim=1), average='macro')

    if f1m > best_f1:
        best_f1, noimp = f1m, 0
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, 'best.pth'))
        torch.save({'logits': val_logits, 'labels': val_labs}, os.path.join(OUTPUT_DIR, 'blob.pt'))
    else:
        noimp += 1
        if noimp >= patience:
            print("Early stop")
            break

    print(f"Epoch {epoch}: TrainL={avg_tr:.4f}, ValL={avg_val:.4f}, F1_macro={f1m:.4f}")

# --------------------------
# THRESHOLD & ROC
# --------------------------
blob = torch.load(os.path.join(OUTPUT_DIR, 'blob.pt'))
labels = blob['labels'].numpy()
probs  = blob['logits'].softmax(1)[:, 1].numpy()

prec, rec, thr = precision_recall_curve(labels, probs)
best_thr = thr[np.argmin(np.abs(prec - rec))]
with open(os.path.join(OUTPUT_DIR, 'thr.txt'), 'w') as f:
    f.write(str(best_thr))

fpr, tpr, _ = roc_curve(labels, probs)
auc_v = auc(fpr, tpr)

plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, label=f"AUC={auc_v:.3f}")
pt = np.argmin(np.abs(_ - best_thr))
plt.scatter(fpr[pt], tpr[pt], s=50, label=f"thr={best_thr:.3f}")
plt.plot([0,1], [0,1], 'k--', alpha=0.4)
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# --------------------------
# INFERENCE ON TEST
# --------------------------
print("\nRunning inference on test set…")
model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, 'best.pth')))
model.eval()
thr = float(open(os.path.join(OUTPUT_DIR, 'thr.txt')).read())

for img_path, mask_path in test_set.samples:
    img = Image.open(img_path).convert('RGB')
    it  = val_transform(img)
    used = ''
    if mask_path:
        mimg = Image.open(mask_path).convert('L')
        mt   = mask_transform(mimg)
        it  *= mt
        used = '(mask)'

    with torch.no_grad(), autocast(**AMP_KW):
        lg = model(it.unsqueeze(0).to(DEVICE))
    p  = lg.softmax(1)[0,1].item()
    pr = 'bats' if p >= thr else 'background'
    print(f"{os.path.basename(img_path)}: {pr} (P={p:.3f}) {used}")

# --------------------------
# 🧪 FINAL EVALUATION ON TEST SET
# --------------------------
print("\nEvaluating on independent TEST set…")
model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, 'best.pth')))
model.eval()

test_logits, test_labels = [], []
with torch.no_grad():
    for x, y in tqdm(test_loader, desc="Test", unit="batch"):
        x, y = x.to(DEVICE), y.to(DEVICE)
        with autocast(**AMP_KW):
            out = model(x)
        test_logits.append(out.cpu())
        test_labels.append(y.cpu())

test_logits = torch.cat(test_logits)
test_labels = torch.cat(test_labels)
test_probs  = test_logits.softmax(1)[:,1]
test_preds  = (test_probs >= best_thr).int()

precision = precision_score(test_labels, test_preds, average='macro')
recall    = recall_score(test_labels, test_preds, average='macro')
f1        = f1_score(test_labels, test_preds, average='macro')
fpr, tpr, _ = roc_curve(test_labels, test_probs)
roc_auc   = auc(fpr, tpr)

print(
    f"\n📊 TEST SET PERFORMANCE:\n"
    f"Precision: {precision:.4f} │ Recall: {recall:.4f} │ "
    f"F1_macro: {f1:.4f} │ AUC: {roc_auc:.4f}"
)

print("\nClassification Report:\n", classification_report(
    test_labels, test_preds, target_names=test_set.classes
))

plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, label=f"ROC AUC = {roc_auc:.3f}")
plt.plot([0,1], [0,1], 'k--', alpha=0.3)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - TEST set")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()
